# Mehanizmi pažnje i transformeri

Jedan od glavnih nedostataka rekurentnih mreža je taj što sve riječi u nizu imaju isti utjecaj na rezultat. To uzrokuje suboptimalne performanse kod standardnih LSTM encoder-decoder modela za zadatke prevođenja nizova, poput prepoznavanja imenovanih entiteta i strojnog prevođenja. U stvarnosti, određene riječi u ulaznom nizu često imaju veći utjecaj na izlazne nizove od drugih.

Razmotrimo model prevođenja nizova, poput strojnog prevođenja. On se implementira pomoću dvije rekurentne mreže, gdje jedna mreža (**encoder**) sažima ulazni niz u skriveno stanje, a druga mreža, **decoder**, razvija to skriveno stanje u prevedeni rezultat. Problem s ovim pristupom je što završno stanje mreže teško pamti početak rečenice, što uzrokuje lošu kvalitetu modela kod dugih rečenica.

**Mehanizmi pažnje** omogućuju ponderiranje kontekstualnog utjecaja svakog ulaznog vektora na svaku izlaznu predikciju RNN-a. To se implementira stvaranjem prečaca između međustanja ulaznog RNN-a i izlaznog RNN-a. Na taj način, prilikom generiranja izlaznog simbola $y_t$, uzimamo u obzir sva skrivena stanja ulaza $h_i$, s različitim težinskim koeficijentima $\alpha_{t,i}$.

![Slika koja prikazuje encoder/decoder model s aditivnim slojem pažnje](../../../../../translated_images/hr/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder model s mehanizmom aditivne pažnje iz [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citirano iz [ovog blog posta](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matrica pažnje $\{\alpha_{i,j}\}$ predstavlja stupanj u kojem određene ulazne riječi sudjeluju u generiranju određene riječi u izlaznom nizu. Ispod je primjer takve matrice:

![Slika koja prikazuje uzorak poravnanja pronađen od strane RNNsearch-50, preuzeto iz Bahdanau - arviz.org](../../../../../translated_images/hr/bahdanau-fig3.09ba2d37f202a6af.webp)

*Slika preuzeta iz [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Slika 3)*

Mehanizmi pažnje odgovorni su za velik dio trenutnog ili gotovo trenutnog stanja tehnike u obradi prirodnog jezika. Međutim, dodavanje pažnje značajno povećava broj parametara modela, što je dovelo do problema skaliranja s RNN-ovima. Ključno ograničenje skaliranja RNN-ova je to što rekurentna priroda modela otežava grupiranje i paralelizaciju treninga. U RNN-u svaki element niza mora se obraditi redoslijedom, što znači da se ne može lako paralelizirati.

Usvajanje mehanizama pažnje u kombinaciji s ovim ograničenjem dovelo je do stvaranja sadašnjih modela transformera, koji su trenutno stanje tehnike, a koje danas poznajemo i koristimo, od BERT-a do OpenGPT3.

## Modeli transformera

Umjesto prosljeđivanja konteksta svake prethodne predikcije u sljedeći korak evaluacije, **modeli transformera** koriste **pozicijske kodove** i pažnju kako bi uhvatili kontekst danog ulaza unutar zadanog prozora teksta. Slika ispod prikazuje kako pozicijski kodovi s pažnjom mogu uhvatiti kontekst unutar zadanog prozora.

![Animirani GIF koji prikazuje kako se evaluacije provode u modelima transformera.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Budući da se svaka ulazna pozicija neovisno mapira na svaku izlaznu poziciju, transformeri se mogu bolje paralelizirati od RNN-ova, što omogućuje mnogo veće i izražajnije jezične modele. Svaka glava pažnje može se koristiti za učenje različitih odnosa između riječi, što poboljšava zadatke obrade prirodnog jezika.

**BERT** (Bidirectional Encoder Representations from Transformers) je vrlo velika višeslojna mreža transformera s 12 slojeva za *BERT-base* i 24 za *BERT-large*. Model se prvo unaprijed trenira na velikom korpusu tekstualnih podataka (WikiPedia + knjige) koristeći nenadzirano učenje (predviđanje maskiranih riječi u rečenici). Tijekom unaprijed treniranja model usvaja značajnu razinu razumijevanja jezika, koja se zatim može iskoristiti s drugim skupovima podataka pomoću finog podešavanja. Ovaj proces naziva se **prijenosno učenje**.

![Slika s http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/hr/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Postoji mnogo varijacija arhitektura transformera, uključujući BERT, DistilBERT, BigBird, OpenGPT3 i druge, koje se mogu fino podešavati. Paket [HuggingFace](https://github.com/huggingface/) pruža repozitorij za treniranje mnogih od ovih arhitektura s PyTorchom.

## Korištenje BERT-a za klasifikaciju teksta

Pogledajmo kako možemo koristiti unaprijed trenirani BERT model za rješavanje našeg tradicionalnog zadatka: klasifikacije nizova. Klasificirat ćemo naš izvorni AG News skup podataka.

Prvo, učitajmo HuggingFace biblioteku i naš skup podataka:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Budući da ćemo koristiti unaprijed trenirani BERT model, trebamo koristiti specifičan tokenizer. Prvo ćemo učitati tokenizer povezan s unaprijed treniranim BERT modelom.

Biblioteka HuggingFace sadrži repozitorij unaprijed treniranih modela, koje možete koristiti jednostavno tako da navedete njihova imena kao argumente funkcijama `from_pretrained`. Svi potrebni binarni fajlovi za model automatski će se preuzeti.

Međutim, ponekad ćete trebati učitati vlastite modele, u kojem slučaju možete navesti direktorij koji sadrži sve relevantne datoteke, uključujući parametre za tokenizer, datoteku `config.json` s parametrima modela, binarne težine itd.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objekt `tokenizer` sadrži funkciju `encode` koja se može izravno koristiti za kodiranje teksta:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Zatim, kreirajmo iteratore koje ćemo koristiti tijekom treninga za pristup podacima. Budući da BERT koristi svoju vlastitu funkciju kodiranja, trebali bismo definirati funkciju za popunjavanje sličnu `padify` koju smo ranije definirali:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

U našem slučaju, koristit ćemo unaprijed istrenirani BERT model nazvan `bert-base-uncased`. Učitajmo model koristeći paket `BertForSequenceClassfication`. Ovo osigurava da naš model već ima potrebnu arhitekturu za klasifikaciju, uključujući završni klasifikator. Vidjet ćete poruku upozorenja koja navodi da težine završnog klasifikatora nisu inicijalizirane i da bi model zahtijevao prethodno treniranje - to je potpuno u redu, jer upravo to namjeravamo učiniti!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Sada smo spremni započeti treniranje! Budući da je BERT već prethodno istreniran, želimo započeti s prilično malom stopom učenja kako ne bismo uništili početne težine.

Sav težak posao obavlja model `BertForSequenceClassification`. Kada pozovemo model na podatke za treniranje, on vraća i gubitak (loss) i izlaz mreže za ulazni minibatch. Gubitak koristimo za optimizaciju parametara (`loss.backward()` provodi povratno širenje), a `out` koristimo za izračun točnosti treniranja uspoređujući dobivene oznake `labs` (izračunate pomoću `argmax`) s očekivanim `labels`.

Kako bismo kontrolirali proces, akumuliramo gubitak i točnost kroz nekoliko iteracija te ih ispisujemo svakih `report_freq` ciklusa treniranja.

Ovo treniranje će vjerojatno trajati dosta dugo, pa ograničavamo broj iteracija.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Možete vidjeti (posebno ako povećate broj iteracija i pričekate dovoljno dugo) da BERT klasifikacija daje prilično dobru točnost! To je zato što BERT već prilično dobro razumije strukturu jezika, a mi samo trebamo fino podesiti završni klasifikator. Međutim, budući da je BERT veliki model, cijeli proces treniranja traje dugo i zahtijeva ozbiljnu računalnu snagu! (GPU, i po mogućnosti više od jednog).

> **Note:** U našem primjeru koristili smo jedan od najmanjih unaprijed treniranih BERT modela. Postoje veći modeli koji vjerojatno daju bolje rezultate.


## Procjena performansi modela

Sada možemo procijeniti performanse našeg modela na testnom skupu podataka. Petlja za evaluaciju je prilično slična petlji za treniranje, ali ne smijemo zaboraviti prebaciti model u način evaluacije pozivom `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Ključne točke

U ovoj jedinici vidjeli smo koliko je jednostavno uzeti unaprijed trenirani jezični model iz biblioteke **transformers** i prilagoditi ga našem zadatku klasifikacije teksta. Slično tome, BERT modeli mogu se koristiti za ekstrakciju entiteta, odgovaranje na pitanja i druge NLP zadatke.

Transformers modeli predstavljaju trenutno najnaprednije stanje u NLP-u, i u većini slučajeva trebali bi biti prvo rješenje s kojim započinjete eksperimentiranje prilikom implementacije prilagođenih NLP rješenja. Međutim, razumijevanje osnovnih temeljnih principa rekurentnih neuronskih mreža, o kojima smo raspravljali u ovom modulu, iznimno je važno ako želite izgraditi napredne neuronske modele.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za nesporazume ili pogrešne interpretacije koje mogu proizaći iz korištenja ovog prijevoda.
